## Ejemplo Práctico 01 - Crear BBDD

Vamos a hacer efectivo el Diagrama de Entidad-Relación en nuestra Base de Datos:

- Toda entidad se transforma en una tabla.

- Todo atributo simple o derivado se transforma en una columna de una tabla.

- Los atributos estructurados transforman los campos en los que se componen en nuevas columnas de la tabla.

- El identificador único de cada entidad se convierte en clave primaria

- Los atributos multivaluados generan una nueva tabla con tres columnas: un id, el id de la tabla de la que surgen propagado como clave ajena y el valor del campo multivaluado.

- Toda relación N_M se transforma en una tabla que tendrá como clave primaria la concatenación de los atributos clave de las entidades que relaciona.

- En la transformación de las relaciones 1:N existen dos posibilidades:

- Transformarlo en una tabla. Se hace como si se tratara de una relación N:M. Es conveniente realizarlo así en caso de que se prevea que en un futuro la relación puede transformarse en N:M y cuando la relación tiene atributos propios.

- Propagar la clave. Se propaga el atributo principal de la entidad que tiene de cardinalidad máxima 1 a la que tiene cardinalidad máxima N, haciendo desaparecer a la relación.

- En la transformación de relaciones 1:1:

- Propagar la clave. Si una de las entidades posee cardinalidad (0,1) y la otra (1,1), se propaga la clave de la entidad con cardinalidad (1,1) a la tabla resultante de la entidad cardinalidad (0,1).

- Si ambas poseen cardinalidades (1,1) se puede propagar la clave de cualquiera de ellas a la tabla resultante de la otra.

- Para los atributos de las relaciones existen dos casos:

- Si la relación es 1:N, sus atributos se propagan a la tabla de lado N, junto con la clave del lado 1

- Si la relación es N:M, sus atributos se transforman en columnas de la tabla generada por dicha relación.

___


Vamos a poner de ejemplo simple una entidad inicial 'persona' que es lo que se nos ocurrió después de pensar un momento, con los siguientes atributos:

|Columnas                            |
|------------------------------------|
|Nombre (**name**)                   |
|Color de los ojos (**eye_color**)   |
|Fecha de nacimiento (**birth_date**)|
|Dirección (**address**)             |
|Comida favorita (**favorite_food**) |

Aunque esta sea una lista básica, nos vale para nuestro propósito. El siguiente paso es asignar nombres de columna y tipos de datos. Hagamos nuestro intento inicial:

|Columna          |Tipo        |Valores permitidos|
|-----------------|------------|------------------|
|**name**         |VARCHAR(40) |                  |
|**eye_color**    |CHAR(2)     |BL, BR, GR        |
|**birth_date**   |DATE        |                  |
|**address**      |VARCHAR(100)|                  |
|**favorite_food**|VARCHAR(200)|                  |

Las columnas name, address y favorite_food son de tipo _**VARCHAR**_ y permiten la entrada de datos de forma libre. La columna eye_color permite dos caracteres que deben corresponder solo a BR, BL o GR (BRown, BLue y GReen respectivamente). La columna birth_date es del tipo _**DATE**_ ya que no necesitamos la hora del nacimiento.
___

### Refinamiento

Ahora vamos con el proceso de normalización para comprobar que no existan columnas duplicadas. Al revisar las columnas en la tabla persona por segunda vez vemos algunos problemas:

- La columna **name** es en realidad un objeto compuesto que consta de nombre y un apellido.

- Dado que es factible que varias personas tengan el mismo nombre, mismo color de ojos, fecha de nacimiento, etc… no hay columnas en la tabla que garanticen la unicidad.

- La columna **address** también es un objeto compuesto de calle, ciudad, estado/provincia, país y código postal.

- La columna de **favorite_food** es una lista que contiene cero o más elementos independientes, sería mejor crear una tabla separada para estos datos que incluya una foreign key a la tabla de persona para que sepa a qué persona se le puede atribuir una comida en particular.

Después de tener en cuenta estos aspectos, la siguiente tabla ofrece una versión normalizada de la tabla de persona:
 
|Columna        |Tipo                                        |Valores permitidos|
|---------------|--------------------------------------------|------------------|
|**persona_id** |SMALLINT UNSIGNED PRIMARY KEY AUTO_INCREMENT|                  |
|**first_name** |VARCHAR(20)                                 |                  |
|**last_name**  |VARCHAR(20)                                 |                  |
|**eye_color**  |CHAR(2)                                     |BR, BL, GR        |
|**birth_date** |DATE                                        |                  |
|**street**     |VARCHAR(20)                                 |                  |
|**city**       |VARCHAR(20)                                 |                  |
|**state**      |VARCHAR(20)                                 |                  |
|**country**    |VARCHAR(20)                                 |                  |
|**postal_code**|VARCHAR(20)                                 |                  |


Ahora que la tabla persona tiene una primary key (**persona_id**) para garantizar la unicidad, el siguiente paso es crear una tabla **favorite_food** que incluya una foreign key para la tabla de persona:

|Columna       |Tipo             |
|--------------|-----------------|
|**persona_id**|SMALLINT UNSIGNED|
|**food**      |VARCHAR(20)      |

- Crear una base de datos (ejecutar el siguiente código en MySQL):

```mysql
/*Crear una base de datos*/

 CREATE DATABASE IF NOT EXISTS personas;

 USE personas;
 
/*Crear la tabla persona*/

 CREATE TABLE persona (
    persona_id SMALLINT UNSIGNED AUTO_INCREMENT,
    fname VARCHAR(20),
    lname VARCHAR(20),
    eye_color ENUM('BR','BL','GR'),
    birth_date DATE,
    street VARCHAR(30),
    city VARCHAR(20),
    state VARCHAR(20),
    country VARCHAR(20),
    postal_code VARCHAR(20),
    PRIMARY KEY (persona_id)
    );
    
/* Ver información de la tabla*/

DESC persona; 
    
```
___

Vamos a hacer un pequeño inciso en nuestro ejemplo para aprender cómo eliminar una Base de Datos o una tabla (en caso de que nos hayamos equivocado) con DROP, IF EXISTS es opcional y lo pondremos en caso de no estar seguros si la base de datos o tabla existe para que no lance error:

```mysql

/*Ejecutar si se quiere eliminar la base de datos o la tabla*/

 DROP DATABASE IF EXISTS personas  -- En caso de querer borrar la base de datos

 DROP TABLE IF EXISTS persona  -- Para borrar una tabla
```

___

Ahora que hemos creado la tabla de persona, el próximo paso será crear la tabla de **favorite_food**:

```mysql

/*Creamos la tabla de comida
  Esta tabla tiene dos columnas como PRIMARY KEY*/

 CREATE TABLE favorite_food (
     persona_id SMALLINT UNSIGNED,
     food VARCHAR(20),
     PRIMARY KEY (persona_id, food),
     FOREIGN KEY (persona_id) REFERENCES persona (persona_id)
     );
     
 DESC favorite_food;
     
```

In [ ]:
################################################################################################################################